<a href="https://colab.research.google.com/github/Azrael-phx/Python__projects/blob/main/python_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
pip install keras-tuner

In [101]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import Sequential

In [102]:
fashion_mnist=keras.datasets.fashion_mnist

In [103]:
(train_images, train_labels),(test_images, test_labels)=fashion_mnist.load_data()

In [104]:
train_images=train_images/255.0
test_images=test_images/255.0

In [105]:
train_images[0].shape

(28, 28)

In [106]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [107]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('Conv_1_filter', min_value = 32, max_value = 128, step = 16),
                                kernel_size = hp.Choice('conv_1_kerne', values = [3,5]),
                                activation = 'relu',
                                input_shape = (28,28,1)
                            ),
                            keras.layers.Conv2D(
                               filters=hp.Int('Conv_2_filter', min_value = 32, max_value = 128, step = 16), 
                               kernel_size = hp.Choice('conv_2_kerne', values = [3,5]),
                                activation = 'relu',
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
                                activation = 'relu',
                            ),
                            keras.layers.Dense(10, activation='softmax')  #Output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate',values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [108]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [109]:
tuner_search=RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,directory='output',project_name="Mnist Fashion")

In [111]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 6 Complete [00h 19m 22s]
val_accuracy: 0.9151666760444641

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 01h 14m 25s
INFO:tensorflow:Oracle triggered exit


In [113]:
model=tuner_search.get_best_models(num_models=1)[0]

In [114]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,015,706
Trainable params: 3,015,706
Non-trainable params: 0
_________________________________________________________________
